<a href="https://colab.research.google.com/github/tejasri2233/FMML_COURSE_ASSIGMENT/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**Question** **1**

Averaging validation accuracy across multiple splits, typically through techniques like k-fold cross-validation, can provide more consistent and reliable results. This helps reduce the impact of randomness in the data split and provides a more robust estimate of your model's performance. It's a common practice in machine learning to assess model performance and gain a better understanding of how well your algorithm generalizes to different subsets of the data.
1. **Reducing Variance**: A single train-test split may not be representative of the entire dataset. By averaging results from multiple splits, you reduce the impact of random variations in data distribution that can affect model performance. This gives you a more stable estimate of your model's generalization ability.

2. **Improved Confidence**: Averaging results provides a confidence interval around your performance metric. You get a clearer sense of how well your model is likely to perform on unseen data, along with an indication of the variability in performance.

3. **Bias Reduction**: A single split might accidentally contain a particularly easy or difficult subset of data. Averaging over multiple splits helps mitigate this bias, ensuring a more accurate assessment of your model's true performance.

4. **Hyperparameter Tuning**: When tuning hyperparameters, using cross-validation allows you to assess how different settings impact model performance more effectively. This can lead to better choices for hyperparameters.

5. **Model Selection**: Cross-validation can aid in comparing different models or algorithms more reliably. You can choose the one with the best average performance across multiple splits.
Averaging validation accuracy across multiple splits is a best practice in machine learning for obtaining a more robust and trustworthy evaluation of your model's performance. It helps you make better-informed decisions about model selection, hyperparameter tuning, and understanding how your model generalizes to unseen data.

Question 2

Averaging validation accuracy across multiple splits, such as through k-fold cross-validation, provides a more accurate estimate of how well your model is likely to perform on unseen data. However, it's important to note that this estimated performance is still not the same as the true test accuracy, which is only obtained when the model is evaluated on a completely independent and previously unseen dataset.

Here's why cross-validation provides a more accurate estimate:

1. **Reduces Overfitting to Validation Data**: When you split your data into a training set and a single validation set, there's a risk that your model might overfit to the specific characteristics of that validation set. Cross-validation mitigates this by averaging results over multiple validation sets, ensuring that the model's performance is not overly influenced by any one particular split.

2. **Generalizes Better**: Cross-validation provides a more generalized estimate of your model's performance because it assesses how well the model performs on various subsets of the data. This makes it a better indicator of how your model will perform on unseen data.

3. **Statistical Significance**: By aggregating results over multiple splits, you can calculate statistical measures like confidence intervals, which give you a better understanding of the range within which the true test accuracy is likely to fall.

While cross-validation gives you a more accurate estimate of test performance, it's still important to eventually evaluate your model on a completely independent test dataset to get the true test accuracy. This dataset should not have been used in any way during the model training or validation processes to ensure an unbiased assessment of your model's performance.

Question 3

The number of iterations or folds in cross-validation can have an impact on the estimate of your model's performance, but there's a trade-off involved.

1. **Fewer Iterations (Folds):**
   - Pros: Faster to compute, less computationally intensive.
   - Cons: Less robust estimate. It may be more influenced by the specific data splits, making it less reliable. It might not capture the variability in your data effectively.

2. **More Iterations (Folds):**
   - Pros: Provides a more robust and reliable estimate of model performance. Reduces the influence of random data splits. Better captures the variability in your data.
   - Cons: Requires more computational resources and time.

In general, increasing the number of iterations (folds) tends to provide a better estimate of your model's performance. However, there are diminishing returns. Beyond a certain point, increasing the number of iterations might not significantly improve the estimate, but it will increase the computational cost.

A common choice is to use k-fold cross-validation, where "k" is typically set between 5 and 10. This strikes a balance between computational cost and obtaining a reasonably robust estimate. However, the choice of "k" can depend on factors like the size of your dataset and the computational resources available.

In summary, increasing the number of iterations in cross-validation can lead to a better estimate of your model's performance, but you should choose a value that balances accuracy with computational resources.

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression  # Replace with your chosen model
from sklearn.datasets import load_iris  # Replace with your dataset

# Load your dataset (replace 'load_iris' with your data loading code)
data = load_iris()
X, y = data.data, data.target

# Create a model (replace with your model creation code)
model = LogisticRegression()

# Specify the number of folds
k = 5

# Create a KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=42)  # 'shuffle' for random data splits

# Perform cross-validation and get accuracy scores
scores = cross_val_score(model, X, y, cv=kf)

# Print the accuracy scores for each fold and the average
for i, score in enumerate(scores):
    print(f"Fold {i + 1} Accuracy: {score:.4f}")

print(f"Average Accuracy: {scores.mean():.4f}")

Fold 1 Accuracy: 1.0000
Fold 2 Accuracy: 1.0000
Fold 3 Accuracy: 0.9333
Fold 4 Accuracy: 0.9667
Fold 5 Accuracy: 0.9667
Average Accuracy: 0.9733


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [16]:

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression  # Replace with your chosen model
from sklearn.datasets import load_iris  # Replace with your dataset

# Load your dataset (replace 'load_iris' with your data loading code)
data = load_iris()
X, y = data.data, data.target

# Create a model (replace with your model creation code)
model = LogisticRegression()

# Specify the number of folds
k = 5

# Create a KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=42)  # 'shuffle' for random data splits

# Perform cross-validation and get accuracy scores
scores = cross_val_score(model, X, y, cv=kf)

# Print the accuracy scores for each fold and the average
for i, score in enumerate(scores):
    print(f"Fold {i + 1} Accuracy: {score:.4f}")

print(f"Average Accuracy: {scores.mean():.4f}")



Fold 1 Accuracy: 1.0000
Fold 2 Accuracy: 1.0000
Fold 3 Accuracy: 0.9333
Fold 4 Accuracy: 0.9667
Fold 5 Accuracy: 0.9667
Average Accuracy: 0.9733


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In this code:

1. We load a dataset (you should replace `load_iris` with your own data loading code).
2. We create a machine learning model (replace `LogisticRegression()` with your chosen model).
3. We specify the number of folds (in this case, 5).
4. We create a `KFold` object, which will handle the data splitting.
5. We use `cross_val_score` to perform cross-validation and obtain accuracy scores for each fold.
6. Finally, we print the accuracy scores for each fold and the average accuracy across all folds.

You can replace the dataset, model, and other parameters to adapt this code to your specific machine learning task.

Question 4

 Increasing the number of iterations (folds) in cross-validation can help to some extent when dealing with a very small training dataset or validation dataset, but it may not completely overcome the limitations imposed by small data. Here's how increasing iterations can help and the caveats to consider:

**Advantages of Increasing Iterations:**
1. **More Reliable Estimates:** With more iterations, you reduce the impact of random data splits. This means your estimate of model performance becomes more reliable and less dependent on the specific data partition.

2. **Better Utilization:** When you have a small dataset, increasing the number of iterations allows you to use the available data more effectively for both training and validation.

**Caveats and Considerations:**
1. **Computationally Expensive:** As you increase the number of iterations, the computational cost also increases. This can be a limiting factor if you have resource constraints.

2. **Limited Data:** While more iterations can improve reliability, they don't magically generate more data. If your dataset is extremely small, cross-validation can only do so much. It's always ideal to have a reasonably sized dataset for robust model training and evaluation.

3. **Risk of Overfitting:** With very small datasets, there's a risk that your model may overfit to the specific data splits in cross-validation, especially if you use a complex model. In such cases, even with more iterations, your model may not generalize well to unseen data.

4. **Bias-Variance Trade-off:** Increasing iterations can help reduce the variance in your estimate, but it doesn't address potential bias introduced by a small dataset. The choice of model and features can have a substantial impact on bias.

In summary, while increasing iterations in cross-validation is a useful strategy to improve reliability and make the best use of available data, it cannot completely compensate for the limitations of an extremely small training or validation dataset. It's crucial to exercise caution and consider the trade-offs involved, and in cases of severely limited data, exploring techniques like data augmentation or acquiring more data may be necessary to build a robust model.

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression  # Replace with your chosen model
import numpy as np

# Example small dataset (replace with your own data)
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
y = np.array([0, 1, 0, 1, 0])

# Create a model (replace with your model creation code)
model = LogisticRegression()

# Specify the number of folds
k = 5  # You can adjust this based on the size of your dataset

# Create a KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=42)  # 'shuffle' for random data splits

# Perform cross-validation and get accuracy scores
scores = cross_val_score(model, X, y, cv=kf)

# Print the accuracy scores for each fold and the average
for i, score in enumerate(scores):
    print(f"Fold {i + 1} Accuracy: {score:.4f}")

print(f"Average Accuracy: {scores.mean():.4f}")

Fold 1 Accuracy: 0.0000
Fold 2 Accuracy: 0.0000
Fold 3 Accuracy: 1.0000
Fold 4 Accuracy: 0.0000
Fold 5 Accuracy: 0.0000
Average Accuracy: 0.2000


In [18]:

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression  # Replace with your chosen model
import numpy as np

# Example small dataset (replace with your own data)
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
y = np.array([0, 1, 0, 1, 0])

# Create a model (replace with your model creation code)
model = LogisticRegression()

# Specify the number of folds
k = 5  # You can adjust this based on the size of your dataset

# Create a KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=42)  # 'shuffle' for random data splits

# Perform cross-validation and get accuracy scores
scores = cross_val_score(model, X, y, cv=kf)

# Print the accuracy scores for each fold and the average
for i, score in enumerate(scores):
    print(f"Fold {i + 1} Accuracy: {score:.4f}")

print(f"Average Accuracy: {scores.mean():.4f}")


Fold 1 Accuracy: 0.0000
Fold 2 Accuracy: 0.0000
Fold 3 Accuracy: 1.0000
Fold 4 Accuracy: 0.0000
Fold 5 Accuracy: 0.0000
Average Accuracy: 0.2000


In this code:

1. We create an example small dataset (`X` and `y`). Replace this with your actual data.

2. We create a machine learning model (replace `LogisticRegression()` with your chosen model).

3. We specify the number of folds (in this case, 5). You can adjust this based on the size of your dataset, but be cautious about using too many folds with very limited data.

4. We create a `KFold` object, which will handle the data splitting.

5. We use `cross_val_score` to perform cross-validation and obtain accuracy scores for each fold.

6. Finally, we print the accuracy scores for each fold and the average accuracy across all folds.

Please replace the dataset and model with your specific data and model of interest.